In [0]:
-- Feature Store Avaliacao

with tb_base as (
  SELECT v.idVendedor,
         ip.idPedido,
         p.dtPedido,
         ap.idAvaliacao,
         ap.vlNota
  from silver.olist.vendendor as v
  LEFT JOIN silver.olist.item_pedido as ip ON v.idVendedor = ip.idVendedor
  LEFT JOIN silver.olist.pedido as p ON ip.idPedido = p.idPedido
  LEFT JOIN silver.olist.avaliacao_pedido as ap ON ip.idPedido = ap.idPedido
  WHERE p.dtPedido < '2017-06-01'
),

-- Incluir os atributos da feature store na tabela
tb_feat_avaliacao AS(
  SELECT idVendedor,
    AVG(vlNota) as mediaAvaliacao,
    COUNT(vlNota) as qtReviewsAteHoje,
    AVG(CASE WHEN dtPedido > date_add('2017-06-01', -28) THEN vlNota END) as mediaAvaliacao28d,
    COUNT(CASE WHEN dtPedido > date_add('2017-06-01', -28) THEN vlNota END) as qtReviewsAteHoje28d,
    try_divide(COUNT(CASE WHEN idAvaliacao IS NULL THEN 1 END) * 1.0, COUNT(idPedido)) as pctPedidosNaoAvaliados,
    ROUND(SUM(CASE WHEN vlNota = 0 THEN 1 ELSE 0 END),1) as qtdeNota0,
    ROUND(SUM(CASE WHEN vlNota = 5 THEN 1 ELSE 0 END),1) as qtdeNota5,
    ROUND(SUM(CASE WHEN vlNota = 4 THEN 1 ELSE 0 END),1) as qtdeNota4,
    ROUND(SUM(CASE WHEN vlNota = 3 THEN 1 ELSE 0 END),1) as qtdeNota3,
    ROUND(SUM(CASE WHEN vlNota = 2 THEN 1 ELSE 0 END),1) as qtdeNota2,
    ROUND(SUM(CASE WHEN vlNota = 1 THEN 1 ELSE 0 END),1) as qtdeNota1,
    ROUND(try_divide(SUM(CASE WHEN vlNota = 0 THEN 1 ELSE 0 END), COUNT(CASE WHEN idAvaliacao IS NOT NULL THEN idAvaliacao END)),1) as pctNota0,
    ROUND(try_divide(SUM(CASE WHEN vlNota = 5 THEN 1 ELSE 0 END), COUNT(CASE WHEN idAvaliacao IS NOT NULL THEN idAvaliacao END)),1) as pctNota5,
    ROUND(try_divide(SUM(CASE WHEN vlNota = 4 THEN 1 ELSE 0 END), COUNT(CASE WHEN idAvaliacao IS NOT NULL THEN idAvaliacao END)),1) as pctNota4,
    ROUND(try_divide(SUM(CASE WHEN vlNota = 3 THEN 1 ELSE 0 END), COUNT(CASE WHEN idAvaliacao IS NOT NULL THEN idAvaliacao END)),1) as pctNota3,
    ROUND(try_divide(SUM(CASE WHEN vlNota = 2 THEN 1 ELSE 0 END), COUNT(CASE WHEN idAvaliacao IS NOT NULL THEN idAvaliacao END)),1) as pctNota2,
    ROUND(try_divide(SUM(CASE WHEN vlNota = 1 THEN 1 ELSE 0 END), COUNT(CASE WHEN idAvaliacao IS NOT NULL THEN idAvaliacao END)),1) as pctNota1
  FROM tb_base
  GROUP BY idVendedor
)

SELECT 
    '2017-06-01' as dtRef,
    *
FROM tb_feat_avaliacao